# 2024衆議院選挙における世論地図のデータ

In [ ]:
# Import necessary libraries
%pip install --upgrade pip jupyter
%pip install pandas
import pandas as pd

In [18]:
# zid(conversation id)と人間向けの表記の対応づけ

zid_to_topic = {
    26: "政治資金規制改革",
    28: "税制",
    29: "社会保障（年金・医療）",
    30: "経済政策・物価高対策",
    31: "エネルギー政策",
    34: "憲法",
    38: "デジタル民主主義"
}
topic_to_zid = {
    "政治資金規制改革": 26,
    "税制": 28,
    "社会保障（年金・医療）": 29,
    "経済政策・物価高対策": 30,
    "エネルギー政策": 31,
    "憲法": 34,
    "デジタル民主主義": 38
}

In [19]:
# 投票データを読み込み、投票行列に変える
df = pd.read_csv("votes.csv")
matrix = df.pivot_table(index="pid", columns=["zid", "tid"], values="vote")
matrix

zid    26                                      28       ...   38            \
tid    0    1    2    3    4    5    6    7    0    1   ...   10   11   12   
pid                                                     ...                  
0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1     1.0  1.0  1.0  1.0 -1.0 -1.0  1.0 -1.0  1.0  1.0  ... -1.0  1.0 -1.0   
2     0.0 -1.0 -1.0 -1.0  1.0  1.0 -1.0 -1.0  0.0  0.0  ...  1.0 -1.0 -1.0   
3     1.0  0.0  0.0 -1.0  1.0  1.0 -1.0 -1.0  1.0  1.0  ...  1.0  1.0 -1.0   
4     1.0 -1.0 -1.0 -1.0  1.0  1.0  0.0 -1.0  0.0 -1.0  ... -1.0 -1.0 -1.0   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
4398  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
4399  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
4400  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
4401  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
4402  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   

zid                                      
tid    13   14   15   16   17   18   19  
pid                                      
0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1    -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  
2    -1.0 -1.0 -1.0  1.0  1.0 -1.0 -1.0  
3    -1.0 -1.0 -1.0 -1.0  1.0 -1.0 -1.0  
4    -1.0 -1.0  0.0  1.0 -1.0 -1.0  1.0  
...   ...  ...  ...  ...  ...  ...  ...  
4398  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
4399  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
4400  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
4401  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
4402  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[4403 rows x 84 columns]

In [20]:
# コメントデータを読み込む
comment_df = pd.read_csv("comments.csv")[["tid", "zid", "txt"]]
comment_df


,tid,zid,txt
0,0,38,行政手続きをすべてオンライン化する
1,0,26,政治資金パーティーはオンライン含めて開催を禁止する
2,1,26,企業・団体によるパーティー券購入は禁止する
3,2,26,企業・団体献金は禁止する
4,3,26,政策活動費は即時廃止する
...,...,...,...
79,15,38,マイナンバーによる行政のデータ連携を加速し全銀行口座との紐づけを実現することで、所得のリアル...
80,16,38,警察によるウェアラブルカメラの運用を広げ、治安維持対応を強化する
81,17,38,行政手続きにAIを導入し、補助金申請の審査業務などは人間からAIに代替させていく
82,18,38,年末調整は廃止し、国民全員がワンストップで確定申告が可能な行政インフラを整える


In [21]:
# 特定のトピックを抽出する例
matrix_keizai = matrix[topic_to_zid["経済政策・物価高対策"]]
matrix_keizai

tid,0,1,2,3,4,5,6
pid,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0
2,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0
3,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0
4,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0
...,...,...,...,...,...,...,...
4398,NaN,NaN,NaN,NaN,-1.0,NaN,NaN
4399,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0
4400,0.0,-1.0,0.0,1.0,0.0,-1.0,1.0


In [22]:
# 特定のトピックのコメントを抽出する例
comment_keizai = comment_df[comment_df["zid"] == topic_to_zid["経済政策・物価高対策"]]
comment_keizai

,tid,zid,txt
38,0,30,低所得者世帯を下支えする給付金を支給する
39,1,30,電気・ガス代、燃料費高騰対策のため、政府による補助金施策を継続
40,2,30,最低賃金を1500円以上に引き上げる
41,3,30,物価高対策として、消費税を引き下げる、又は廃止する
42,4,30,トリガー条項凍結又は二重課税廃止によるガソリン減税により、ガソリン代を引き下げる
43,5,30,再エネ賦課金の徴収停止により電気代を引き下げる
44,6,30,中小企業の価格転嫁を支援し、賃上げを実現する


In [23]:
import pandas as pd

def count_votes(marrix):
    # 集計コード
    # 要注意: Polisの内部データ表現においては賛成が-1
    result = pd.DataFrame({
        'count_agree': (marrix == -1).sum(axis=0),
        'count_neutral': (marrix == 0).sum(axis=0),
        'count_disagree': (marrix == 1).sum(axis=0),
        'count_pass': marrix.isna().sum(axis=0)
    })
    return result

result = count_votes(matrix_keizai)
# 結果を表示
result


,count_agree,count_neutral,count_disagree,count_pass
tid,,,,
0,1351,1162,1111,698
1,2536,693,433,663
2,1961,1158,577,625
3,2040,700,949,648
4,2405,965,323,649
5,2102,1104,431,690
6,2900,617,166,672


In [24]:
# 各tidに対応するtxtを取得し、resultに新しい列として追加
result["txt"] = result.index.map(lambda tid: comment_keizai.loc[comment_keizai["tid"] == tid, "txt"].values[0])
result

,count_agree,count_neutral,count_disagree,count_pass,txt
tid,,,,,
0,1351,1162,1111,698,低所得者世帯を下支えする給付金を支給する
1,2536,693,433,663,電気・ガス代、燃料費高騰対策のため、政府による補助金施策を継続
2,1961,1158,577,625,最低賃金を1500円以上に引き上げる
3,2040,700,949,648,物価高対策として、消費税を引き下げる、又は廃止する
4,2405,965,323,649,トリガー条項凍結又は二重課税廃止によるガソリン減税により、ガソリン代を引き下げる
5,2102,1104,431,690,再エネ賦課金の徴収停止により電気代を引き下げる
6,2900,617,166,672,中小企業の価格転嫁を支援し、賃上げを実現する


In [25]:
# 有効投票(count_agree, count_neutral, count_disagree)の合計を計算
total_counts = result[['count_agree', 'count_neutral', 'count_disagree']].sum(axis=1)
result['N'] = total_counts

# 各値を合計で割ってパーセントに変換
result['賛成(%)'] = (result['count_agree'] / total_counts * 100).round(1)
result['中立(%)'] = (result['count_neutral'] / total_counts * 100).round(1)
result['反対(%)'] = (result['count_disagree'] / total_counts * 100).round(1)

# パーセント表記の列を表示
result[['txt', '賛成(%)', '中立(%)', '反対(%)', 'N']]


,txt,賛成(%),中立(%),反対(%),N
tid,,,,,
0,低所得者世帯を下支えする給付金を支給する,37.3,32.1,30.7,3624
1,電気・ガス代、燃料費高騰対策のため、政府による補助金施策を継続,69.3,18.9,11.8,3662
2,最低賃金を1500円以上に引き上げる,53.1,31.3,15.6,3696
3,物価高対策として、消費税を引き下げる、又は廃止する,55.3,19.0,25.7,3689
4,トリガー条項凍結又は二重課税廃止によるガソリン減税により、ガソリン代を引き下げる,65.1,26.1,8.7,3693
5,再エネ賦課金の徴収停止により電気代を引き下げる,57.8,30.4,11.9,3637
6,中小企業の価格転嫁を支援し、賃上げを実現する,78.7,16.8,4.5,3683


# ここまでの処理を全部まとめたもの

In [26]:
from IPython.display import Markdown, display

for topic in topic_to_zid:
    display(Markdown(f"# {topic}"))
    zid = topic_to_zid[topic]
    m = matrix[zid]
    c = comment_df[comment_df["zid"] == zid]
    result = count_votes(m)
    result["txt"] = result.index.map(lambda tid: c.loc[c["tid"] == tid, "txt"].values[0])

    # 有効投票(count_agree, count_neutral, count_disagree)の合計を計算
    total_counts = result[['count_agree', 'count_neutral', 'count_disagree']].sum(axis=1)
    result['N'] = total_counts

    # 各値を合計で割ってパーセントに変換
    result['賛成(%)'] = (result['count_agree'] / total_counts * 100).round(1)
    result['中立(%)'] = (result['count_neutral'] / total_counts * 100).round(1)
    result['反対(%)'] = (result['count_disagree'] / total_counts * 100).round(1)

    # パーセント表記の列を表示
    display(result[['txt', '賛成(%)', '中立(%)', '反対(%)', 'N']])



# 政治資金規制改革

,txt,賛成(%),中立(%),反対(%),N
tid,,,,,
0,政治資金パーティーはオンライン含めて開催を禁止する,61.1,22.8,16.1,1030
1,企業・団体によるパーティー券購入は禁止する,69.5,19.3,11.2,1029
2,企業・団体献金は禁止する,63.0,20.7,16.3,1029
3,政策活動費は即時廃止する,48.5,31.0,20.5,1029
4,政策活動費は使途公開の義務などをもって実施する,84.3,7.1,8.5,1021
5,政策活動費を維持するにあたり、その監査に関する「第三者機関」を創設する,80.1,12.1,7.8,1032
6,政策活動費に限らず、すべての政治資金を監査する独立の「第三者機関」を創設する,83.3,10.9,5.8,1024
7,調査研究広報滞在費の使途は公開し、未使用分は返還義務を課す,90.8,6.4,2.8,1031


# 税制

,txt,賛成(%),中立(%),反対(%),N
tid,,,,,
0,当面の間、または賃金上昇率が物価＋2％に達するまでの間、消費税は10％から5％に引き下げる,61.8,15.4,22.8,1065
1,消費税を8%とし、軽減税率を廃止する,45.0,28.4,26.6,1069
2,消費税は廃止する,28.3,20.5,51.2,1060
3,所得税を課す最低金額の引き上げを行う。具体的には、基礎控除等の合計を103万円から178万円...,79.3,11.8,8.9,1067
4,働く若者をサポートするため、若者減税（所得税・住民税を減免）を導入する,65.0,18.0,17.0,1074
5,給付（負の所得税）と所得税の還付を組み合わせた新制度「給付付き税額控除」を導入し、基礎的所得...,46.0,37.9,16.0,1060
6,賃上げ減税を拡充する（税額控除額の引上げ、価格転嫁等の取引条件改善企業等への減税等）,70.7,22.9,6.3,1046
7,年末調整制度を見直し、全員確定申告制度導入を検討,29.9,33.6,36.6,1067
8,インボイス制度の廃止,44.7,35.9,19.4,1063


# 社会保障（年金・医療）

...

,txt,賛成(%),中立(%),反対(%),N
tid,,,,,
0,2050年までのカーボンニュートラルの実現,56.4,28.0,15.6,915
1,原子力発電は最大限活用,51.8,18.5,29.7,919
2,化石燃料にも原子力発電にも依存しないカーボンニュートラルを目指し、再生可能エネルギーによる発...,39.7,26.4,33.9,915
3,原子力発電所の新増設は認めない,40.2,19.8,40.1,916
4,原子力規制委員会が策定した世界で最も厳しい基準を満たした原子力発電所の再稼働を認める,66.4,12.5,21.1,915
5,可能な限り原発依存度を低減し、将来的に原発に依存しない社会を目指す,54.1,15.8,30.1,904
6,「核のごみ」問題を解決しうる核融合発電を柱に据えて技術開発を推進,71.5,18.3,10.2,908
7,再生可能エネルギー推進による国内外へのバラマキと環境破壊を阻止する,64.4,23.4,12.2,902


# 憲法

,txt,賛成(%),中立(%),反対(%),N
tid,,,,,
0,憲法改正を積極的に議論する,59.1,21.9,19.0,1311
1,現行の9条を維持し、自衛隊を明記の上、自衛の措置（自衛権）についても言及する,62.0,18.0,20.0,1320
2,緊急事態対応：緊急事態においても、国会の機能をできるだけ維持する。それが難しい場合、内閣の権...,47.3,22.0,30.7,1326
3,参議院の合区を解消し、各都道府県から必ず1人以上選出するよう定める,42.5,38.8,18.7,1314
4,教育の重要性を国の理念として位置づけ、国民誰もがその機会を享受できるようにする。私学助成の規...,68.7,22.9,8.4,1303
5,憲法53条後段には、衆議院か参議院のいずれかの議院の総議員の4分の1以上の要求があれば、内閣...,58.5,35.7,5.9,1307
6,衆議院の解散権について、内閣が恣意的にタイミングを選べるような運用は是正されるべきであり、こ...,48.1,32.2,19.7,1303
7,デジタル社会において一人一人が自律的な個人として尊重される人権保障のあり方を具体的に検討する...,72.6,19.5,8.0,1305
8,オンラインによる国会審議、採決に参加できる制度を創設,57.2,22.2,20.6,1299


# デジタル民主主義

,txt,賛成(%),中立(%),反対(%),N
tid,,,,,
0,行政手続きをすべてオンライン化する,70.5,15.0,14.5,234
1,個人情報保護法を欧州並みに強化する,48.0,29.1,22.9,227
2,インターネット投票の導入,66.2,8.8,25.0,228
3,デジタル技術を活用したプッシュ型の行政サービス支援,79.0,17.2,3.9,233
4,行政のデジタルトランスフォーメーション（DX）を推進する,90.1,8.2,1.7,233
5,自己情報コントロール権の保護を強化する,72.1,21.0,6.9,233
6,インターネットの広告規制：ターゲット広告やなりすまし広告に対する規制を強化する,89.6,7.0,3.5,230
7,デジタルインフラの拡充：通信基盤の強化やクラウドインフラの国内拡充を進める,89.5,6.1,4.4,228
8,地方創生にデジタル技術を活用する,90.9,8.7,0.4,230
